In [1]:
import pandas as pd
from ajuste import ajustar_precos as ajustar_precos
import chart_studio
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go

# username = 'XXX'
# api_key = 'YYYY' 
# chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [3]:
# Histórico dos preços de fechamento do HGLG11
hglg11_precos = pd.read_csv('data/hglg11_precos.csv', parse_dates=['data'])
hglg11_precos

,data,preco
0,2011-03-02,104.40
1,2011-03-21,104.40
2,2011-03-28,104.40
3,2011-03-31,104.40
4,2011-04-05,104.40
...,...,...
2767,2022-12-05,162.84
2768,2022-12-06,162.95
2769,2022-12-07,162.18
2770,2022-12-08,162.20


In [5]:
# Histórico de dividendos do HGLG11 em que a data base foi ajustada
# para ser necessariamente em dia com pregão
hglg11_divs = (
    pd.read_csv('data/hglg11_dividendos.csv', parse_dates=['data_com'])
    # Remover as colunas que não serão utilizadas
    .drop(columns=['tipo', 'data_pagamento'])
)
hglg11_divs

,data_com,valor_div
0,2011-03-31,0.72
1,2011-04-29,0.75
2,2011-05-31,0.79
3,2011-06-30,0.80
4,2011-07-29,0.81
...,...,...
136,2022-07-29,1.10
137,2022-08-31,1.10
138,2022-09-30,1.10
139,2022-10-31,1.10


In [6]:
ajustar_precos(hglg11_precos, hglg11_divs)
hglg11_precos = ajustar_precos(hglg11_precos, hglg11_divs)
hglg11_precos

,data,preco,preco_aj,valor_div
0,2011-03-02,104.40,37.450552,NaN
1,2011-03-21,104.40,37.450552,NaN
2,2011-03-28,104.40,37.450552,NaN
3,2011-03-31,104.40,37.450552,0.72
4,2011-04-05,104.40,37.710625,NaN
...,...,...,...,...
2767,2022-12-05,162.84,162.840000,NaN
2768,2022-12-06,162.95,162.950000,NaN
2769,2022-12-07,162.18,162.180000,NaN
2770,2022-12-08,162.20,162.200000,NaN


In [7]:
# Calcular o dividend_yield
hglg11_precos["dividend_yield(%)"] = 100 * hglg11_precos["valor_div"] / hglg11_precos["preco"]
hglg11_precos.query("valor_div > 0")

,data,preco,preco_aj,valor_div,dividend_yield(%)
3,2011-03-31,104.40,37.450552,0.72,0.689655
13,2011-04-27,104.40,37.710625,0.75,0.718391
27,2011-05-31,104.30,37.947112,0.79,0.757430
39,2011-06-30,106.30,38.969935,0.80,0.752587
48,2011-07-21,109.00,40.262777,0.81,0.743119
...,...,...,...,...,...
2680,2022-07-29,168.64,163.207107,1.10,0.652277
2703,2022-08-31,172.49,168.029090,1.10,0.637718
2724,2022-09-30,171.50,168.136933,1.10,0.641399
2744,2022-10-31,165.39,163.193471,1.10,0.665095


In [8]:
hglg11_precos.query('data >= "2022-11-27"')

,data,preco,preco_aj,valor_div,dividend_yield(%)
2762,2022-11-28,162.90,161.812748,NaN,NaN
2763,2022-11-29,163.50,162.408743,NaN,NaN
2764,2022-11-30,164.81,163.710000,1.1,0.667435
2765,2022-12-01,162.71,162.710000,NaN,NaN
2766,2022-12-02,162.80,162.800000,NaN,NaN
2767,2022-12-05,162.84,162.840000,NaN,NaN
2768,2022-12-06,162.95,162.950000,NaN,NaN
2769,2022-12-07,162.18,162.180000,NaN,NaN
2770,2022-12-08,162.20,162.200000,NaN,NaN
2771,2022-12-09,162.97,162.970000,NaN,NaN


In [26]:
# Comparar os preços com e sem ajuste do HGLG11 com o Plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco"],
        mode="lines",
        line=dict(color="#97c9cc"),
        name="Preço sem ajuste",        
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco_aj"],
        mode="lines",
        line=dict(color="#02878e"),
        name="Preço com ajuste",
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento do HGLG11 <br>(com e sem ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()
# py.plot(fig, filename = 'hglg_petr_1', auto_open=True)

In [10]:
# Valorização do HGLG11 com e sem ajuste de dividendos
valorizacao_sa = (hglg11_precos["preco"].iloc[-1] / hglg11_precos["preco"].iloc[0] - 1) * 100
valorizacao_ca = (hglg11_precos["preco_aj"].iloc[-1] / hglg11_precos["preco_aj"].iloc[0] - 1) * 100
print(f"Valorização do HGLG11 com ajuste de dividendos: {valorizacao_sa:.2f}%")
print(f"Valorização do HGLG11 sem ajuste de dividendos: {valorizacao_ca:.2f}%")

Valorização do HGLG11 com ajuste de dividendos: 56.10%
Valorização do HGLG11 sem ajuste de dividendos: 335.16%


In [11]:
# Carregar os dados de preços e dividendos do BBSA3
bbas3_precos = (
    pd.read_csv('data/bbas3_precos.csv', parse_dates=['data'])
    # Concidir com o período de negociação do HGLG11
    .query('data >= "2011-03-02"')
    # Renomear com o preço desejado para "preco"
    .rename(columns={'preult': 'preco'})
    # Selecionar apenas as colunas de interesse
    [['data', 'preco']]
)
bbas3_precos

,data,preco
41,2011-03-02,29.180000
42,2011-03-03,29.540001
43,2011-03-04,29.620001
44,2011-03-09,29.620001
45,2011-03-10,28.010000
...,...,...
2964,2022-12-12,33.790001
2965,2022-12-13,31.860001
2966,2022-12-14,31.070000
2967,2022-12-15,31.950001


In [12]:
bbas3_divs = pd.read_csv('data/bbas3_dividendos.csv', parse_dates=['data_com'])
bbas3_divs[-5:]

,data_com,valor_div
96,2022-05-23,0.673071
97,2022-06-13,0.250285
98,2022-08-22,0.787938
99,2022-09-12,0.273736
100,2022-11-21,0.804684


In [13]:
bbas3_divs.query('data_com.dt.year == 2022').valor_div.sum()

3.8250640999999996

In [14]:
# Ajustart os preços de fechamento da BBSA3
bbas3_precos = ajustar_precos(bbas3_precos, bbas3_divs)
bbas3_precos

,data,preco,preco_aj,valor_div
0,2011-03-02,29.180000,13.311282,NaN
1,2011-03-03,29.540001,13.475507,NaN
2,2011-03-04,29.620001,13.512001,NaN
3,2011-03-09,29.620001,13.512001,NaN
4,2011-03-10,28.010000,12.777553,NaN
...,...,...,...,...
2923,2022-12-12,33.790001,33.790001,NaN
2924,2022-12-13,31.860001,31.860001,NaN
2925,2022-12-14,31.070000,31.070000,NaN
2926,2022-12-15,31.950001,31.950001,NaN


In [15]:
# Calcular o dividend_yield
bbas3_precos["dividend_yield(%)"] = 100 * bbas3_precos["valor_div"] / bbas3_precos["preco"]
bbas3_precos.query("valor_div > 0")

,data,preco,preco_aj,valor_div,dividend_yield(%)
12,2011-03-22,29.000000,13.229170,0.253055,0.872602
52,2011-05-19,27.250000,12.540285,0.156962,0.576006
75,2011-06-21,27.049999,12.520363,0.257515,0.951995
116,2011-08-18,25.200001,11.776182,0.211485,0.839225
139,2011-09-21,25.100000,11.828720,0.278181,1.108291
...,...,...,...,...,...
2783,2022-05-23,38.560001,35.808551,0.673071,1.745516
2798,2022-06-13,33.759998,31.908012,0.250285,0.741366
2847,2022-08-22,41.470001,39.487814,0.787938,1.900020
2861,2022-09-12,41.180000,39.971134,0.273736,0.664729


In [24]:
# Plotar os preços da BBSA3 e HGLG11 sem ajustes
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=bbas3_precos["data"],
        y=bbas3_precos["preco"],
        mode="lines",
        name="BBSA3",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
    )
)
# Adicionar os pontos de dividendos da BBSA3 no próprio gráfico de preços
fig.add_trace(
    go.Scatter(
        x=bbas3_precos.query('valor_div > 0').data,
        y=bbas3_precos.query('valor_div > 0')["preco"],
        mode="markers",
        name="Dividendos da BBSA3",
        marker=dict(
            color="black",
            size=(30 * bbas3_precos.query('valor_div > 0')["dividend_yield(%)"]) ** 0.5,
            symbol="circle"),
    )
)
# Adicionar os pontos de dividendos da HGLG11 no próprio gráfico de preços, sendo que o tamanho dos pontos é proporcional ao valor do dividendo
fig.add_trace(
    go.Scatter(
        x=hglg11_precos.query('valor_div > 0').data,
        y=hglg11_precos.query('valor_div > 0')["preco"],
        mode="markers",
        name="Dividendos do HGLG11",
        marker=dict(
            color="black",
            size=(30 * hglg11_precos.query('valor_div > 0')["dividend_yield(%)"]) ** 0.5,
            symbol="circle"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br> (sem ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()
# py.plot(fig, filename = 'hglg_petr_2', auto_open=True)

In [18]:
# Adicionar uma coluna com o preço normalizado em bbas3_precos e hglg11_precos
bbas3_precos["preco_normalizado"] = 100 * bbas3_precos["preco"] / bbas3_precos["preco"].iloc[0]
hglg11_precos["preco_normalizado"] = 100 * hglg11_precos["preco"] / hglg11_precos["preco"].iloc[0]

In [19]:
# Comparar os preços ajustados e normalizados (base 100) para o início da série dos dois ativos com o Plotly
import plotly.express as px

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=bbas3_precos["data"],
        y=bbas3_precos["preco_normalizado"],
        mode="lines",
        name="BBSA3",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco_normalizado"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
    )
)
fig.update_layout(
    # mudar a fonte
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br>(normalizados e sem ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()
# py.plot(fig, filename = 'hglg_petr_3', auto_open=True)

In [20]:
bbas3_precos.query('valor_div > 0')

,data,preco,preco_aj,valor_div,dividend_yield(%),preco_normalizado
12,2011-03-22,29.000000,13.229170,0.253055,0.872602,99.383139
52,2011-05-19,27.250000,12.540285,0.156962,0.576006,93.385881
75,2011-06-21,27.049999,12.520363,0.257515,0.951995,92.700476
116,2011-08-18,25.200001,11.776182,0.211485,0.839225,86.360524
139,2011-09-21,25.100000,11.828720,0.278181,1.108291,86.017820
...,...,...,...,...,...,...
2783,2022-05-23,38.560001,35.808551,0.673071,1.745516,132.145308
2798,2022-06-13,33.759998,31.908012,0.250285,0.741366,115.695675
2847,2022-08-22,41.470001,39.487814,0.787938,1.900020,142.117892
2861,2022-09-12,41.180000,39.971134,0.273736,0.664729,141.124058


In [21]:
# Adicionar uma coluna com o preço ajustado e normalizado em bbas3_precos e hglg11_precos
bbas3_precos["preco_aj_normalizado"] = 100 * bbas3_precos["preco_aj"] / bbas3_precos["preco_aj"].iloc[0]
hglg11_precos["preco_aj_normalizado"] = 100 * hglg11_precos["preco_aj"] / hglg11_precos["preco_aj"].iloc[0]

In [25]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=bbas3_precos["data"],
        y=bbas3_precos["preco_aj_normalizado"],
        mode="lines",
        name="BBSA3",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco_aj_normalizado"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
        
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br> (normalizados e com ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()
# py.plot(fig, filename = 'hglg_petr_4', auto_open=True)